<a href="https://colab.research.google.com/github/maricamolesi/Iniciacao-Cientifica/blob/master/Rede_Neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Neste notebook foi utilizado o banco de imagens Brodatz 128x128 contendo 1776 imagens de textura, com leve corrosão (pelo código dos Automâtos Celulares; parâmetros v = 1, gamma = 0.01 e 30 iterações). O conjunto já estava dividido em pastas de treino e validação, feito na proporção meio a meio de forma aleatória.


In [31]:
# License: BSD
# Author: Sasank Chilamkurthy
# Modified: Mariana Camolesi

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torch.utils import data
import random
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode


In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
}

data_dir = '/content/drive/My Drive/ProjetoIC/trainval_v10gamma001int30'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=10, shuffle=True, num_workers=5) 
                for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
           

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model

In [68]:
#Finetuning the convnet
#Load a pretrained model (Resnet50) and reset final fully connected layer.

model_ft = models.resnet50(pretrained=True)

num_ftrs = model_ft.fc.in_features

# The size of each output sample is len(class_names)
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# All parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.002, momentum=0.9)

# Decay LR by a factor of 0.5 every 9 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=9, gamma=0.5)



In [ ]:
#Train and evaluate
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

Epoch 0/9
----------
train Loss: 0.1209 Acc: 0.9752
val Loss: 0.0950 Acc: 0.9775

Epoch 1/9
----------
train Loss: 0.1809 Acc: 0.9583
val Loss: 0.0966 Acc: 0.9764

Epoch 2/9
----------
train Loss: 0.1275 Acc: 0.9730
val Loss: 0.0975 Acc: 0.9707

Epoch 3/9
----------
train Loss: 0.1105 Acc: 0.9775
val Loss: 0.0884 Acc: 0.9775

Epoch 4/9
----------
train Loss: 0.1163 Acc: 0.9730
val Loss: 0.0855 Acc: 0.9809

Epoch 5/9
----------
train Loss: 0.1249 Acc: 0.9696
val Loss: 0.0956 Acc: 0.9752

Epoch 6/9
----------
train Loss: 0.1250 Acc: 0.9741
val Loss: 0.0865 Acc: 0.9786

Epoch 7/9
----------
train Loss: 0.1064 Acc: 0.9809
val Loss: 0.0894 Acc: 0.9809

Epoch 8/9
----------
train Loss: 0.1202 Acc: 0.9775
val Loss: 0.0866 Acc: 0.9786

Epoch 9/9
----------
train Loss: 0.1160 Acc: 0.9741
val Loss: 0.0916 Acc: 0.9752

Training complete in 5m 40s
Best val Acc: 0.980856


In [ ]:
#Save trained model
torch.save(model_ft, "/content/drive/My Drive/ProjetoIC/Rede-Neural.pt")